In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn.model_selection import train_test_split

from keras.callbacks import EarlyStopping

2023-06-18 14:11:58.935198: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-18 14:11:59.003679: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-18 14:11:59.004970: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-18 14:12:02.960023: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
csv = pd.read_csv(
    "/home/brijesh/Documents/ai/spam-detection/datasets/yt.csv",
    delimiter=","
)
df = csv[["Comment", "Spam"]]

df = df.dropna()


In [4]:

X = df['Comment'].values
y = df['Spam'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
    
print(len(X_train))
print(len(X_test))

model = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(model, input_shape=[], dtype=tf.string, trainable=True)

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

print(model.summary())

11792
2948


2023-06-18 14:12:22.825153: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-18 14:12:22.826650: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-06-18 14:12:22.914094: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 192762400 exceeds 10% of free system memory.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dense (Dense)               (None, 16)                816       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________
None


2023-06-18 14:12:24.076982: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'sentences' with dtype string and shape [?]
	 [[{{node sentences}}]]
2023-06-18 14:12:24.082664: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder' with dtype string and shape [?]
	 [[{{node Placeholder}}]]


In [5]:


early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# fit the model
model.fit(X_train,
         y_train,
         epochs=40,
         batch_size=512,
         validation_data=(X_test, y_test),
         verbose=1,
         callbacks=[early_stop]
)


model.save("trained-model.h5")

Epoch 1/40


2023-06-18 14:12:24.628048: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/StatefulPartitionedCall_grad/StatefulPartitionedCall' with dtype float and shape [?,50]
	 [[{{node gradients/StatefulPartitionedCall_grad/StatefulPartitionedCall}}]]
2023-06-18 14:12:24.628130: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/StatefulPartitionedCall_grad/StatefulPartitionedCall_3' with dtype int64 and shape [?]
	 [[{{node gradients/StatefulPartitionedCall_grad/StatefulPartitionedCall_3}}]]
2023-06-18 14:12:24.634943: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 192762400 exceeds 10% of f

 1/24 [>.............................] - ETA: 1:21 - loss: 0.6736 - accuracy: 0.5938

2023-06-18 14:12:28.927591: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 192762400 exceeds 10% of free system memory.


24/24 [==============================] - 13s 400ms/step - loss: 0.5219 - accuracy: 0.9176 - val_loss: 0.3991 - val_accuracy: 0.9732
Epoch 2/40
24/24 [==============================] - 8s 342ms/step - loss: 0.3317 - accuracy: 0.9736 - val_loss: 0.2587 - val_accuracy: 0.9766
Epoch 3/40
24/24 [==============================] - 8s 335ms/step - loss: 0.2291 - accuracy: 0.9759 - val_loss: 0.1849 - val_accuracy: 0.9780
Epoch 4/40
24/24 [==============================] - 8s 336ms/step - loss: 0.1793 - accuracy: 0.9763 - val_loss: 0.1512 - val_accuracy: 0.9786
Epoch 5/40
24/24 [==============================] - 8s 331ms/step - loss: 0.1566 - accuracy: 0.9766 - val_loss: 0.1337 - val_accuracy: 0.9790
Epoch 6/40
24/24 [==============================] - 8s 323ms/step - loss: 0.1432 - accuracy: 0.9766 - val_loss: 0.1222 - val_accuracy: 0.9790
Epoch 7/40
24/24 [==============================] - 8s 325ms/step - loss: 0.1331 - accuracy: 0.9766 - val_loss: 0.1138 - val_accuracy: 0.9790
Epoch 8/40
24/24

In [6]:
# model_prod = tf.keras.models.load_model('./trained-model.h5', custom_objects={'KerasLayer':hub.KerasLayer})


In [9]:
for test in ["Congratulations! You have been selected to receive a free gift worth $1000. Claim your prize by clicking this link and entering your personal information. Hurry, this offer is only available for a limited time!","good video"]:
    out = model.predict([test])
    print(out)

1/1 [==============================] - 0s 277ms/step
[[0.00585569]]
1/1 [==============================] - 0s 125ms/step
[[2.5443951e-05]]
